In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras as K
import keras.layers as ll
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

%matplotlib inline

In [57]:
df = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

In [58]:
vocab = {}
i = 1
max_len = 0
for word in df['Word']:
    max_len = len(word) if len(word)>max_len else max_len
    if word not in vocab:
        vocab[word] = i
        i += 1
for word in df_test['Word']:
    max_len = len(word) if len(word)>max_len else max_len
    if word not in vocab:
        vocab[word] = i
        i += 1
max_len

38

In [59]:
model = K.Sequential(name='Surname_classifier')

model.add(ll.Embedding(len(vocab), 512, input_length=1))

model.add(ll.Dense(256))
model.add(ll.BatchNormalization())
model.add(ll.Activation('sigmoid'))

model.add(ll.Dense(128))
model.add(ll.BatchNormalization())
model.add(ll.Activation('sigmoid'))

model.add(ll.Dense(64))
model.add(ll.BatchNormalization())
model.add(ll.Activation('sigmoid'))

model.add(ll.Flatten())
model.add(ll.Dense(1))
model.add(ll.BatchNormalization())
model.add(ll.Activation('sigmoid'))

model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1, 512)            145534464 
_________________________________________________________________
dense_30 (Dense)             (None, 1, 256)            131328    
_________________________________________________________________
batch_normalization_30 (Batc (None, 1, 256)            1024      
_________________________________________________________________
activation_30 (Activation)   (None, 1, 256)            0         
_________________________________________________________________
dense_31 (Dense)             (None, 1, 128)            32896     
_________________________________________________________________
batch_normalization_31 (Batc (None, 1, 128)            512       
_________________________________________________________________
activation_31 (Activation)   (None, 1, 128)            0         
__________

In [61]:
df['Rank'] = df['Word'].map(vocab)
df_test['Rank'] = df_test['Word'].map(vocab)

In [62]:
X_train, X_val, y_train, y_val = train_test_split(df['Rank'], df['Label'], stratify=df['Label'])

In [63]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((76056,), (25352,), (76056,), (25352,))

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1)

/home/pingvindex/apps/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 145534464 elements. This may consume a large amount of memory.
  "This may consume a large amount of memory." % num_elements)


Train on 76056 samples, validate on 25352 samples
Epoch 1/1


In [ ]:
y_pred = model.predict(X_val)

In [ ]:
ytr_pred = model.predict(X_train)

In [49]:
ras = roc_auc_score(y_val, y_pred)
ras_tr = roc_auc_score(y_train, ytr_pred)

0.0065685753